In [1]:
import os

In [2]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer'

In [54]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    all_required_files_in_main_catalog: list
    library_dataset_name: str
    all_required_files_in_train_test_validations_catalogs: list

In [55]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories


In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            all_required_files_in_main_catalog=config.all_required_files_in_main_catalog,
            library_dataset_name=config.library_dataset_name,
            all_required_files_in_train_test_validations_catalogs=config.all_required_files_in_train_test_validations_catalogs
        )

        return data_validation_config

    

In [57]:
import os
from mlops_NLP_Text_Summarization.logging import logger

In [58]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files_in_main_catalog = os.listdir(os.path.join("artifacts","data_ingestion",f"{self.config.library_dataset_name}"))
            files_in_train_test_validations_catalogs = os.listdir(os.path.join("artifacts","data_ingestion",f"{self.config.library_dataset_name}","train"))
            files_in_train_test_validations_catalogs.extend(os.listdir(os.path.join("artifacts","data_ingestion",f"{self.config.library_dataset_name}","test")))
            files_in_train_test_validations_catalogs.extend(os.listdir(os.path.join("artifacts","data_ingestion",f"{self.config.library_dataset_name}","validation")))
            
            for file in all_files_in_main_catalog:
                if file not in self.config.all_required_files_in_main_catalog:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status})")
                    for file in self.config.all_required_files_in_train_test_validations_catalogs:
                        if file not in files_in_train_test_validations_catalogs:
                            validation_status = False
                            with open(self.config.status_file, 'w') as f:
                                f.write(f"Validation status: {validation_status})")
                
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [59]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2023-08-06 21:57:42,903: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-06 21:57:42,904: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-06 21:57:42,906: INFO: common: created directory at: artifacts/data_validation]
